# Gapminder
Inspired by https://www.gapminder.org/tools/

In [ ]:
import pandas as pd
import ahlive as ah

# open and combine dataframes
pop_df = ah.tutorial.open_dataset('gapminder_population')
lex_df = ah.tutorial.open_dataset('gapminder_life_expectancy')
inc_df = ah.tutorial.open_dataset('gapminder_income')
cnt_df = ah.tutorial.open_dataset('gapminder_country')
df = pop_df.merge(lex_df, on=['geo', 'time']).merge(inc_df, on=['geo', 'time'])
df = df.set_index('geo').join(cnt_df.set_index('country')[['name', 'region']])

# subset and preprocess
countries = [
    "United States", "China", "Kenya", 'New Zealand',
    "Germany", "Brazil", "Afghanistan"
]
df = df.loc[df['name'].isin(countries)]
df = df.loc[df['time'].isin([1820, 1920, 2020])]
df['population'] = df['population'] / 1e6

# animate
caption = (
    'Income per person (GDP per capita, PPP$ inflation – adjusted), '
    'Life Expectancy (Years), Population (million)')
ah_df = ah.DataFrame(
    df, 'income', 'life_expectancy', s='population', alpha=0.5,
    label='name', inline_labels='name', state_labels='time',
    title='Income, Life Expectancy, & Population',
    subtitle='Source: World Bank Gapminder',
    note='Population symbolized by size of dots',
    caption=caption, figsize=(12, 7), legend=False, revert='boomerang',
    xlim0s=0, xlim1s='explore', xmargins=0.1, ymargins=0.1,
).remark(
    remarks='s', durations=1.5, state_labels=[1820, 1920, 2020]
).config(
    'remark_inline', suffix='M'
)
ah_df.render()